<a href="https://colab.research.google.com/github/YezhouLi/628-Module-3-Group-9/blob/master/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ijson

     |████████████████████████████████| 61kB 1.7MB/s 


In [0]:
import json as js
import ijson

In [9]:
filename = "/content/drive/My Drive/review.json"
with open(filename, 'r') as f:
    objects = ijson.items(f, 'text')
    columns = list(objects)

IncompleteJSONError: ignored

In [0]:
print(columns[0])